In [1]:
import warnings
import argparse
import DrawingWithTensors
import pandas as pd
from random import randint
import torch
import os
import argparse
import time

# initiate the parser
parser = argparse.ArgumentParser(description = "List of options to run application when creating custom datset")

parser = argparse.ArgumentParser()  
parser.add_argument("-V", "--version", help="show program version", action="store_true")
parser.add_argument("-s", "--size", help="upper bound of number of images to include")
parser.add_argument("-t", "--rotations", help="allow random image and tensor rotations to be made")
parser.add_argument("-r", "--root_folder", help="destination for root folder")
parser.add_argument("-g", "--generation", help="which generation number we are using")
parser.add_argument("-d", "--dimensions", help="square image dimensions")

#defined defaults
upper_bound = 101
isize = 400
iteration = "0"
rotations= False
ROOT = "/home/peo5032/Documents/COMP594/input/gen"


# read arguments from the command line
args = parser.parse_args(['-g','6','-s','80000','-d','416','-t','false'])

# check for --version or -V
if args.version:  
    print("this is version 0.1", flush=True)
    
if args.size: 
    print("will create", upper_bound, "images", flush=True)
    upper_bound = int(args.size) + 1

if args.rotations: 
    rotations= args.rotations.lower() == "true"
    print("rotations was", args.rotations, flush=True)
    
if args.root_folder:  
    os.makedirs(root_folder, exist_ok=True)
    print("destination was", args.root_folder, flush=True)
    
if args.generation:
    print("iteration was", args.generation, flush=True)
    iteration = args.generation
    
if args.dimensions:
    print("dimension chosen was", args.dimensions,flush=True)
    isize = int(args.dimensions)
    
factor = 0.45

os.makedirs(ROOT, exist_ok=True)
IMAGE_PATH = ROOT + iteration + "/roads"
os.makedirs(IMAGE_PATH, exist_ok=True)

TENSOR_PATH = ROOT + iteration + "/tensor_values"
os.makedirs(TENSOR_PATH, exist_ok=True)

PICKLE_PATH = ROOT + iteration

df = pd.DataFrame()
NumLanes = []
ShldrWidth = []
ShldrWidthCenter = []
RoadWidth = []
FileNames = []
imageGen = DrawingWithTensors.datasetFactory(IMAGE_SIZE = isize)
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


will create 101 images
rotations was false
iteration was 6
dimension chosen was 416


In [2]:
tmp_tensor = torch.zeros(1,isize,isize,dtype=torch.float32)
since = time.time()
for i in range(0,upper_bound):
    
    if i % 10 == 0:
        print("Picture ",i, flush=True)
    c = randint(0,80)
    lanecount = randint(1,5)
    laneWidth = randint(17,35)
    lineWidth = randint(1,2)
    shoulderWidth = randint(0,89)
    
    #create tuple of information, img, and tensor
    tuple,img,tmp_tensor = imageGen.generateNewImageWithTensor(c,lanecount,laneWidth,lineWidth,shoulderWidth, tmp_tensor)
    roadWidth,laneCount,shoulderWidth,centerShldrWidth = tuple       
    
    #if(rotations and i % 2 != 0):
    #    img, tmp_tensor = DrawingWithTensors.rotationOfImageAndTensor(img, tmp_tensor, imageGen.classList, randint(0,180))
    
    NumLanes.append(laneCount)
    ShldrWidth.append(shoulderWidth)
    RoadWidth.append (roadWidth)
    ShldrWidthCenter.append(centerShldrWidth)
    
    FileName = str(i) + ".png"
    FileNames.append(FileName)
    img.save(IMAGE_PATH + "/" + FileName,"PNG")
    
    #save tensor
    torch.save(tmp_tensor, TENSOR_PATH + "/"+ str(i) + '.pt')
    
    img.close()
    tmp_tensor = torch.mul(tmp_tensor, 0)

df['NumLanes'] = NumLanes
df['ShldrWidth'] = ShldrWidth
df['RdwyWidth'] = RoadWidth
df['ShldrWidthCenter'] = ShldrWidthCenter
df['FileName'] = FileNames

df.to_pickle(PICKLE_PATH + "/train_images_v2.pkl")

total = time.time() - since
print("program terminated in {:0f}s".format(total), flush=True)

Picture  0
Picture  10
Picture  20
Picture  30
Picture  40
Picture  50
Picture  60
Picture  70
Picture  80
Picture  90
Picture  100
Picture  110
Picture  120
Picture  130
Picture  140
Picture  150
Picture  160
Picture  170
Picture  180
Picture  190
Picture  200
Picture  210
Picture  220
Picture  230
Picture  240
Picture  250
Picture  260
Picture  270
Picture  280
Picture  290
Picture  300
Picture  310
Picture  320
Picture  330
Picture  340
Picture  350
Picture  360
Picture  370
Picture  380
Picture  390
Picture  400
Picture  410
Picture  420
Picture  430
Picture  440
Picture  450
Picture  460
Picture  470
Picture  480
Picture  490
Picture  500
Picture  510
Picture  520
Picture  530
Picture  540
Picture  550
Picture  560
Picture  570
Picture  580
Picture  590
Picture  600
Picture  610
Picture  620
Picture  630
Picture  640
Picture  650
Picture  660
Picture  670
Picture  680
Picture  690
Picture  700
Picture  710
Picture  720
Picture  730
Picture  740
Picture  750
Picture  760
Picture  7

Picture  5940
Picture  5950
Picture  5960
Picture  5970
Picture  5980
Picture  5990
Picture  6000
Picture  6010
Picture  6020
Picture  6030
Picture  6040
Picture  6050
Picture  6060
Picture  6070
Picture  6080
Picture  6090
Picture  6100
Picture  6110
Picture  6120
Picture  6130
Picture  6140
Picture  6150
Picture  6160
Picture  6170
Picture  6180
Picture  6190
Picture  6200
Picture  6210
Picture  6220
Picture  6230
Picture  6240
Picture  6250
Picture  6260
Picture  6270
Picture  6280
Picture  6290
Picture  6300
Picture  6310
Picture  6320
Picture  6330
Picture  6340
Picture  6350
Picture  6360
Picture  6370
Picture  6380
Picture  6390
Picture  6400
Picture  6410
Picture  6420
Picture  6430
Picture  6440
Picture  6450
Picture  6460
Picture  6470
Picture  6480
Picture  6490
Picture  6500
Picture  6510
Picture  6520
Picture  6530
Picture  6540
Picture  6550
Picture  6560
Picture  6570
Picture  6580
Picture  6590
Picture  6600
Picture  6610
Picture  6620
Picture  6630
Picture  6640
Pictur

Picture  11680
Picture  11690
Picture  11700
Picture  11710
Picture  11720
Picture  11730
Picture  11740
Picture  11750
Picture  11760
Picture  11770
Picture  11780
Picture  11790
Picture  11800
Picture  11810
Picture  11820
Picture  11830
Picture  11840
Picture  11850
Picture  11860
Picture  11870
Picture  11880
Picture  11890
Picture  11900
Picture  11910
Picture  11920
Picture  11930
Picture  11940
Picture  11950
Picture  11960
Picture  11970
Picture  11980
Picture  11990
Picture  12000
Picture  12010
Picture  12020
Picture  12030
Picture  12040
Picture  12050
Picture  12060
Picture  12070
Picture  12080
Picture  12090
Picture  12100
Picture  12110
Picture  12120
Picture  12130
Picture  12140
Picture  12150
Picture  12160
Picture  12170
Picture  12180
Picture  12190
Picture  12200
Picture  12210
Picture  12220
Picture  12230
Picture  12240
Picture  12250
Picture  12260
Picture  12270
Picture  12280
Picture  12290
Picture  12300
Picture  12310
Picture  12320
Picture  12330
Picture  1

Picture  17150
Picture  17160
Picture  17170
Picture  17180
Picture  17190
Picture  17200
Picture  17210
Picture  17220
Picture  17230
Picture  17240
Picture  17250
Picture  17260
Picture  17270
Picture  17280
Picture  17290
Picture  17300
Picture  17310
Picture  17320
Picture  17330
Picture  17340
Picture  17350
Picture  17360
Picture  17370
Picture  17380
Picture  17390
Picture  17400
Picture  17410
Picture  17420
Picture  17430
Picture  17440
Picture  17450
Picture  17460
Picture  17470
Picture  17480
Picture  17490
Picture  17500
Picture  17510
Picture  17520
Picture  17530
Picture  17540
Picture  17550
Picture  17560
Picture  17570
Picture  17580
Picture  17590
Picture  17600
Picture  17610
Picture  17620
Picture  17630
Picture  17640
Picture  17650
Picture  17660
Picture  17670
Picture  17680
Picture  17690
Picture  17700
Picture  17710
Picture  17720
Picture  17730
Picture  17740
Picture  17750
Picture  17760
Picture  17770
Picture  17780
Picture  17790
Picture  17800
Picture  1

Picture  22620
Picture  22630
Picture  22640
Picture  22650
Picture  22660
Picture  22670
Picture  22680
Picture  22690
Picture  22700
Picture  22710
Picture  22720
Picture  22730
Picture  22740
Picture  22750
Picture  22760
Picture  22770
Picture  22780
Picture  22790
Picture  22800
Picture  22810
Picture  22820
Picture  22830
Picture  22840
Picture  22850
Picture  22860
Picture  22870
Picture  22880
Picture  22890
Picture  22900
Picture  22910
Picture  22920
Picture  22930
Picture  22940
Picture  22950
Picture  22960
Picture  22970
Picture  22980
Picture  22990
Picture  23000
Picture  23010
Picture  23020
Picture  23030
Picture  23040
Picture  23050
Picture  23060
Picture  23070
Picture  23080
Picture  23090
Picture  23100
Picture  23110
Picture  23120
Picture  23130
Picture  23140
Picture  23150
Picture  23160
Picture  23170
Picture  23180
Picture  23190
Picture  23200
Picture  23210
Picture  23220
Picture  23230
Picture  23240
Picture  23250
Picture  23260
Picture  23270
Picture  2

...

Picture  55440
Picture  55450
Picture  55460
Picture  55470
Picture  55480
Picture  55490
Picture  55500
Picture  55510
Picture  55520
Picture  55530
Picture  55540
Picture  55550
Picture  55560
Picture  55570
Picture  55580
Picture  55590
Picture  55600
Picture  55610
Picture  55620
Picture  55630
Picture  55640
Picture  55650
Picture  55660
Picture  55670
Picture  55680
Picture  55690
Picture  55700
Picture  55710
Picture  55720
Picture  55730
Picture  55740
Picture  55750
Picture  55760
Picture  55770
Picture  55780
Picture  55790
Picture  55800
Picture  55810
Picture  55820
Picture  55830
Picture  55840
Picture  55850
Picture  55860
Picture  55870
Picture  55880
Picture  55890
Picture  55900
Picture  55910
Picture  55920
Picture  55930
Picture  55940
Picture  55950
Picture  55960
Picture  55970
Picture  55980
Picture  55990
Picture  56000
Picture  56010
Picture  56020
Picture  56030
Picture  56040
Picture  56050
Picture  56060
Picture  56070
Picture  56080
Picture  56090
Picture  5

Picture  60910
Picture  60920
Picture  60930
Picture  60940
Picture  60950
Picture  60960
Picture  60970
Picture  60980
Picture  60990
Picture  61000
Picture  61010
Picture  61020
Picture  61030
Picture  61040
Picture  61050
Picture  61060
Picture  61070
Picture  61080
Picture  61090
Picture  61100
Picture  61110
Picture  61120
Picture  61130
Picture  61140
Picture  61150
Picture  61160
Picture  61170
Picture  61180
Picture  61190
Picture  61200
Picture  61210
Picture  61220
Picture  61230
Picture  61240
Picture  61250
Picture  61260
Picture  61270
Picture  61280
Picture  61290
Picture  61300
Picture  61310
Picture  61320
Picture  61330
Picture  61340
Picture  61350
Picture  61360
Picture  61370
Picture  61380
Picture  61390
Picture  61400
Picture  61410
Picture  61420
Picture  61430
Picture  61440
Picture  61450
Picture  61460
Picture  61470
Picture  61480
Picture  61490
Picture  61500
Picture  61510
Picture  61520
Picture  61530
Picture  61540
Picture  61550
Picture  61560
Picture  6

Picture  66380
Picture  66390
Picture  66400
Picture  66410
Picture  66420
Picture  66430
Picture  66440
Picture  66450
Picture  66460
Picture  66470
Picture  66480
Picture  66490
Picture  66500
Picture  66510
Picture  66520
Picture  66530
Picture  66540
Picture  66550
Picture  66560
Picture  66570
Picture  66580
Picture  66590
Picture  66600
Picture  66610
Picture  66620
Picture  66630
Picture  66640
Picture  66650
Picture  66660
Picture  66670
Picture  66680
Picture  66690
Picture  66700
Picture  66710
Picture  66720
Picture  66730
Picture  66740
Picture  66750
Picture  66760
Picture  66770
Picture  66780
Picture  66790
Picture  66800
Picture  66810
Picture  66820
Picture  66830
Picture  66840
Picture  66850
Picture  66860
Picture  66870
Picture  66880
Picture  66890
Picture  66900
Picture  66910
Picture  66920
Picture  66930
Picture  66940
Picture  66950
Picture  66960
Picture  66970
Picture  66980
Picture  66990
Picture  67000
Picture  67010
Picture  67020
Picture  67030
Picture  6

Picture  71850
Picture  71860
Picture  71870
Picture  71880
Picture  71890
Picture  71900
Picture  71910
Picture  71920
Picture  71930
Picture  71940
Picture  71950
Picture  71960
Picture  71970
Picture  71980
Picture  71990
Picture  72000
Picture  72010
Picture  72020
Picture  72030
Picture  72040
Picture  72050
Picture  72060
Picture  72070
Picture  72080
Picture  72090
Picture  72100
Picture  72110
Picture  72120
Picture  72130
Picture  72140
Picture  72150
Picture  72160
Picture  72170
Picture  72180
Picture  72190
Picture  72200
Picture  72210
Picture  72220
Picture  72230
Picture  72240
Picture  72250
Picture  72260
Picture  72270
Picture  72280
Picture  72290
Picture  72300
Picture  72310
Picture  72320
Picture  72330
Picture  72340
Picture  72350
Picture  72360
Picture  72370
Picture  72380
Picture  72390
Picture  72400
Picture  72410
Picture  72420
Picture  72430
Picture  72440
Picture  72450
Picture  72460
Picture  72470
Picture  72480
Picture  72490
Picture  72500
Picture  7

Picture  77320
Picture  77330
Picture  77340
Picture  77350
Picture  77360
Picture  77370
Picture  77380
Picture  77390
Picture  77400
Picture  77410
Picture  77420
Picture  77430
Picture  77440
Picture  77450
Picture  77460
Picture  77470
Picture  77480
Picture  77490
Picture  77500
Picture  77510
Picture  77520
Picture  77530
Picture  77540
Picture  77550
Picture  77560
Picture  77570
Picture  77580
Picture  77590
Picture  77600
Picture  77610
Picture  77620
Picture  77630
Picture  77640
Picture  77650
Picture  77660
Picture  77670
Picture  77680
Picture  77690
Picture  77700
Picture  77710
Picture  77720
Picture  77730
Picture  77740
Picture  77750
Picture  77760
Picture  77770
Picture  77780
Picture  77790
Picture  77800
Picture  77810
Picture  77820
Picture  77830
Picture  77840
Picture  77850
Picture  77860
Picture  77870
Picture  77880
Picture  77890
Picture  77900
Picture  77910
Picture  77920
Picture  77930
Picture  77940
Picture  77950
Picture  77960
Picture  77970
Picture  7

In [3]:
#from PIL import Image
#testing output of data-set
#img = Image.open(IMAGE_PATH+"/500.png")
#test_tensor = torch.load(TENSOR_PATH+'/500.pt')
#img

In [4]:
#DrawingWithTensors.showInferenceOnImage(img, test_tensor, "road", 0.6, imageGen.classMap)